In [1]:
!nvidia-smi

Fri Nov 15 17:28:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  | 00000000:00:0A.0 Off |                  N/A |
| 27%   31C    P8               1W / 260W |      1MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Script to run code on the cluster

In [2]:
import torch
torch.cuda.is_available()

True

In [17]:
!python compute_prs.py --dataset imagenet --device cuda:0 --model ViT-B-32 --pretrained laion2b_s34b_b79k --data_path datasets/imagenet --cache_dir "../cache/" --samples_per_class 3


^C
Traceback (most recent call last):
  File "/cluster/work/vogtlab/Group/vstrozzi/working-MT2024/compute_prs.py", line 10, in <module>
    from utils.factory import create_model_and_transforms, get_tokenizer
  File "/cluster/work/vogtlab/Group/vstrozzi/working-MT2024/utils/__init__.py", line 2, in <module>
    from utils.factory import create_model, create_model_and_transforms, create_model_from_pretrained, get_tokenizer, create_loss
  File "/cluster/work/vogtlab/Group/vstrozzi/working-MT2024/utils/factory.py", line 9, in <module>
    import timm
  File "/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/__init__.py", line 2, in <module>
    from .layers import is_scriptable, is_exportable, set_scriptable, set_exportable
  File "/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/layers/__init__.py", line 8, in <module>
    from .classifier import create_classifier, ClassifierHead, NormMlpClassi

In [12]:
!python compute_text_projection.py  --dataset imagenet --device cuda:0 --model ViT-B-32 --pretrained laion2b_s34b_b79k --cache_dir "../cache/"

/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Using local files
/cluster/work/vogtlab/Group/vstrozzi/working-MT2024/utils/factory.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they 

In [115]:
!python compute_text_set_projection.py --device cuda:0 --model ViT-B-32 --pretrained laion2b_s34b_b79k --cache_dir "../cache/" --data_path text_descriptions/mscoco.txt

/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Using local files
/cluster/work/vogtlab/Group/vstrozzi/working-MT2024/utils/factory.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they 

# Test complete text set

In [11]:
!python compute_complete_text_set.py --device cuda:0 --model ViT-B-32 --texts_per_head 100 --num_of_last_layers 4 --text_descriptions image_descriptions_general

/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Number of layers: 12
  0%|                                                     | 0/4 [00:00<?, ?it/s][3125 2368 1612  856  100]
Epoch [20/1500], Loss: 2.060335
Epoch [40/1500], Loss: 1.363378
Epoch [60/1500], Loss: 0.879052
Epoch [80/1500], Loss: 0.540339
Epoch [100/1500], Loss: 0.310504
Epoch [120/1500], Loss: 0.171811
Epoch [140/1500], Loss: 0.092677
Epoch [160/1500], Loss: 0.050686
Epoch [180/1500], Loss: 0.027356
Epoch [200/1500], Loss: 0.014978
Epoch [220/1500], Loss: 0.008589
Epoch [240/1500], Loss: 0.004211
Epoch [260/1500], Loss: 0.003089
Epoch [280/1500], Loss: 0.003224
Epoch [300/1500], Loss: 0.001741
Usng step side 2368
Epoch [320/1500], Loss: 0.001904
E

In [17]:
!python compute_complete_text_set.py --device cuda:0 --model ViT-B-32 --texts_per_head 100 --num_of_last_layers 4 --text_descriptions image_descriptions_general

/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Number of layers: 12
  0%|                                                     | 0/4 [00:00<?, ?it/s][3125 2368 1612  856  100]
Epoch [20/1500], Loss: 0.517662
Epoch [40/1500], Loss: 0.467849
Epoch [60/1500], Loss: 0.444533
Epoch [80/1500], Loss: 0.417296
Epoch [100/1500], Loss: 0.391512
Epoch [120/1500], Loss: 0.367294
Epoch [140/1500], Loss: 0.342210
Epoch [160/1500], Loss: 0.319005
Epoch [180/1500], Loss: 0.293711
Epoch [200/1500], Loss: 0.272005
Epoch [220/1500], Loss: 0.248405
Epoch [240/1500], Loss: 0.231151
Epoch [260/1500], Loss: 0.209372
Epoch [280/1500], Loss: 0.189782
Epoch [300/1500], Loss: 0.171604
Epoch [320/1500], Loss: 0.146924
Epoch [340/1500], Los

In [132]:
!python compute_complete_text_set.py --device cuda:0 --model ViT-B-32 --texts_per_head 200 --num_of_last_layers 4 --text_descriptions image_descriptions_general

/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Number of layers: 12
  0%|                                                     | 0/4 [00:00<?, ?it/s]Epoch [20/1500], Loss: 2.088930
Epoch [40/1500], Loss: 1.430712
Epoch [60/1500], Loss: 0.972141
Epoch [80/1500], Loss: 0.635344
Epoch [100/1500], Loss: 0.389819
Epoch [120/1500], Loss: 0.227019
Epoch [140/1500], Loss: 0.131881
Epoch [160/1500], Loss: 0.075046
Epoch [180/1500], Loss: 0.042864
Epoch [200/1500], Loss: 0.024015
Epoch [220/1500], Loss: 0.014911
Epoch [240/1500], Loss: 0.007019
Epoch [260/1500], Loss: 0.004476
Epoch [280/1500], Loss: 0.004681
Epoch [300/1500], Loss: 0.002330
Epoch [320/1500], Loss: 0.003541
Epoch [340/1500], Loss: 0.003774
Epoch [360/1500